In [14]:
!pip install torch torchvision torchaudio


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\bvais\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
!pip install sounddevice transformers librosa matplotlib noisereduce


  Using cached sounddevice-0.5.5-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached transformers-5.1.0-py3-none-any.whl.metadata (31 kB)
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached matplotlib-3.10.8-cp311-cp311-win_amd64.whl.metadata (52 kB)
  Using cached noisereduce-3.0.3-py3-none-any.whl.metadata (14 kB)
  Using cached cffi-2.0.0-cp311-cp311-win_amd64.whl.metadata (2.6 kB)
  Using cached huggingface_hub-1.4.1-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.22.2-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
  Using cached typer_slim-0.23.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached numba-0.63.1-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl.metadata (16 kB)
  Using cached pooch-1.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Usi


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\bvais\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
import sounddevice as sd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torch
import noisereduce as nr
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import wave


In [ ]:

# Recording parameters
DURATION = 5          # seconds
SAMPLE_RATE = 16000   # wav2vec2 expects 16kHz

print("🎙️ Recording... Speak now!")
audio = sd.rec(int(DURATION * SAMPLE_RATE),
               samplerate=SAMPLE_RATE,
               channels=1,
               dtype='float32')
sd.wait()
print("✅ Recording finished")

speech = audio.flatten()

  Using cached torch-2.10.0-cp311-cp311-win_amd64.whl.metadata (31 kB)
  Using cached torchvision-0.25.0-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached torchaudio-2.10.0-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached markupsafe-3.0.3-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
Using cached torch-2.10.0-cp311-cp311-win_amd64.whl (113.7 MB)
Using cached torchvision-0.25.0-cp311-cp311-win_amd64.whl (4.0 MB)
Using cached torchaudio-2.10.0-cp311-cp311-win_amd64.whl (474 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached markupsafe-3.0.3-cp311-cp311-win_amd64.whl (15 kB)



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\bvais\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import sounddevice as sd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torch
import noisereduce as nr
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import scipy.io.wavfile as wav


# Recording parameters
DURATION = 5          # seconds
SAMPLE_RATE = 16000   # wav2vec2 expects 16kHz

print("🎙️ Recording... Speak now!")
audio = sd.rec(int(DURATION * SAMPLE_RATE),
               samplerate=SAMPLE_RATE,
               channels=1,
               dtype='float32')
sd.wait()
print("✅ Recording finished")

speech = audio.flatten()


🎙️ Recording... Speak now!
✅ Recording finished


In [11]:
speech_int16 = np.int16(speech * 32767)
wav.write("speech.wav", SAMPLE_RATE, speech_int16)
print("Audio saved successfully!")


Audio saved successfully!


In [12]:
speech_clean = nr.reduce_noise(
    y=speech,
    sr=SAMPLE_RATE
)

print("Noise reduction complete ✅")


Noise reduction complete ✅


In [8]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")


Loading weights: 100%|██████████| 404/404 [00:01<00:00, 274.73it/s, Materializing param=wav2vec2.feature_projection.projection.weight]                         
Wav2Vec2ForCTC LOAD REPORT from: facebook/wav2vec2-large-960h
Key                        | Status  | 
---------------------------+---------+-
wav2vec2.masked_spec_embed | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [13]:
input_values = processor(
    speech_clean,
    sampling_rate=SAMPLE_RATE,
    return_tensors="pt",
    padding=True
).input_values

with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)[0]

print("\n==============================")
print("📝 TRANSCRIPTION OUTPUT:")
print("==============================")
print(transcription)



📝 TRANSCRIPTION OUTPUT:
THERE
